In [51]:
using PowerModels
using Ipopt
using ForwardDiff
using OrdinaryDiffEq
using NLsolve

display(HTML("<style>.container { width:100% !important; }</style>"))

data = PowerModels.parse_file("Twogen2bus.m")
pf_result = run_ac_pf(data ,Ipopt.Optimizer)
#PowerModels.print_summary(pf_result["solution"])

# Load flow results

# bus results
va1 =pf_result["solution"]["bus"]["1"]["va"]
va2 =pf_result["solution"]["bus"]["2"]["va"]
vm1 =pf_result["solution"]["bus"]["1"]["vm"]
vm2 =pf_result["solution"]["bus"]["2"]["vm"];

# Voltage at bus 1 (grid forming)
V_tR = vm1*cos(va1)
V_tI = vm1*sin(va1)


# Voltage at bus 2 (grid following)
V_tR1 = vm2*cos(va2)
V_tI1 = vm2*sin(va2)

# generator results
pg = pf_result["solution"]["gen"]["1"]["pg"]
qg = pf_result["solution"]["gen"]["1"]["qg"]

# branch results
p_ref =pf_result["solution"]["branch"]["1"]["pf"]
q_ref =pf_result["solution"]["branch"]["1"]["qf"]
p_ref1 =pf_result["solution"]["branch"]["1"]["pt"]
q_ref1 =pf_result["solution"]["branch"]["1"]["qt"];

HTML{String}("<style>.container { width:100% !important; }</style>")

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       28
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       40

Total number of variables............................:       10
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       10
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [53]:
PowerModels.print_summary(pf_result["solution"])

Metadata
  baseMVA: 1000
  per_unit: true

Table Counts
  bus: 2
  gen: 1
  branch: 1


Table: bus
            vm,     va
      1: 1.020,  0.000
      2: 1.000, -0.128


Table: gen
            pg,    qg
      1: 0.703, 0.051


Table: branch
            pf,    qf,     pt,     qt
      1: 0.203, 0.001, -0.200, -0.020


In [54]:
PowerModels.print_summary(data)

Metadata
  baseMVA: 1000
  name: Twogen2bus
  per_unit: true
  source_type: matpower
  source_version: 2

Table Counts
  bus: 2
  load: 2
  shunt: 0
  gen: 1
  storage: 0
  switch: 0
  branch: 1
  dcline: 0


Table: bus
         bus_i, bus_type,    vm
      1:     1,        3, 1.020
      2:     2,        1, 1.000

  default values:
    base_kv: 320.000
    va: 0.000
    vmin: 0.940
    vmax: 1.060
    area: 1
    source_id: [(2)]
    zone: 1


Table: load
         load_bus,    pd,    qd
      1:        1, 0.500, 0.050
      2:        2, 0.200, 0.020

  default values:
    status: 1
    source_id: [(2)]


Table: gen
         gen_bus,    pg,    qg,    vg,    mbase,  pmin,  pmax,   qmin,  qmax, gen_status,   pc1,   pc2, qc1max, qc1min, qc2max, qc2min, ramp_10, ramp_30, ramp_agc, ramp_q, source_id
      1:       1, 0.700, 0.070, 1.020, 1000.000, 0.000, 1.000, -0.100, 0.100,       1000, 0.000, 0.000,  0.000,  0.000,  0.000,  0.000,   0.000,   0.000,    0.000,  0.000,     [(2)]


Table: bra

In [55]:
# Define the parameters
lf =0.08
rf = 0.003
cf = 0.074

# Need to find values for these

rt =0.00005
lt = 0.15

Rp = 0.02
Rq =0.001
ωz = 2*pi*5.0


kvp = 0.52
kvi = 1.16
kvf = 1.0
rv = 0.0
lv =0.2
kip = 0.74
kii = 1.19
kif = 0
#kip=1.27
#kii=14.3
#kvp = 0.59
#kvi = 736

ωg = 1.0 #ω_apc 
ω_ref = 1.0
ωb = 2*pi*50.0
ω_star = 1
#qg=0
#pg=0.5
v_ref = V_tR
v_g=V_tR

1.02

In [83]:
lt=0.15

0.15

In [93]:
function system!(F,x)
    pm = x[1]*x[5] + x[2]*x[6]
    qm = -x[1]*x[6] + x[2]*x[5]
    ω_apc = 1
    v_hat = 1.065
    v_iref_d = v_hat - rv*x[5] + ω_apc*lv*x[6]
    v_iref_q = - rv*x[6] - ω_apc*lv*x[5]
    i_hat_d = kvp*(v_iref_d -x[1]) + kvi*x[10] - ω_apc*cf*x[2] + kif*x[5]
    i_hat_q = kvp*(v_iref_d -x[2]) + kvi*x[11] + ω_apc*cf*x[1] + kif*x[6]
    v_md = kip*(i_hat_d -x[3]) + kii*x[12] - ω_apc*lf*x[4] + kvf*x[1]
    v_mq = kip*(i_hat_q -x[4]) + kii*x[13] + ω_apc*lf*x[3] + kvf*x[2]
    v_gd = v_g*cos(x[9])
    v_gq = -v_g*sin(x[9])
    ωg=ω_apc
    
    
### Grid forming equations
  #𝜕eg/𝜕t
F[1]= (ωb / cf)*(x[3] - x[5]) + ωg*ωb*x[2]
  #𝜕eq/𝜕t
F[2]= (ωb / cf)*(x[4] - x[6]) - ωg*ωb*x[1]
   #𝜕isd/𝜕t
F[3]= (ωb / lf)*(v_md - x[1]) - (rf*ωb/lf)*x[3] + ωb*ωg*x[4]
   #𝜕isq/𝜕t
F[4]= (ωb / lf)*(v_mq - x[2]) - (rf*ωb/lf)*x[4] - ωb*ωg*x[3]
     #𝜕igd/𝜕t
F[5]= (ωb / lt)*(x[1] - v_gd) - (rt*ωb/lt)*x[5] + ωb*ωg*x[6]
   #𝜕igq/𝜕t
F[6]= (ωb / lt)*(x[2] - v_gq) - (rt*ωb/lt)*x[6] - ωb*ωg*x[5] 
   #𝜕pf/𝜕t
F[7]=  ωz*(pm- x[7])
   #𝜕qf/𝜕t
F[8]= ωz*(qm - x[8])
  #𝜕δc/𝜕t
F[9]= ωb*(ω_apc-ωg)
   #𝜕ξ_d/𝜕t
F[10]= v_iref_d - x[1]
   #𝜕ξ_q/𝜕t
F[11]= v_iref_q - x[2]
   #𝜕γ_d/𝜕t
F[12]=i_hat_d - x[3]
    #𝜕γ_q/𝜕t
F[13]= i_hat_q - x[4]
    
F[14]= - pg+v_gd*x[5] + v_gq*x[6] 
    
end


#Initial Conditions
      initial =  [
           v_g, #egd1
           0, #egq1
            pg/v_g, #isd1
            qg/v_g, #isq1
            pg/v_g, #igd1
            qg/v_g,#igq1
            pg,
            qg,
            0.2,#δc1
            0.0, #ξ_d1
            0.0, #ξ_q1
            0.0, #γ_d1
            0.0, #γ_q1
            0,
         ]


         sys_solve = NLsolve.nlsolve(
                     system!,
                     initial,
                     method = :trust_region,
                     iterations=1000,
                    ) #Solve using initial guess x0
                      print(sys_solve)


Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [1.02, 0.0, 0.6888242930824501, 0.050321206397434655, 0.6888242930824501, 0.050321206397434655, 0.7026007789440991, 0.05132763052538335, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0]
 * Zero: [1.0241741492535046, -0.13194828644853746, 0.6695056054398789, -0.1283403666877176, 0.6597414322426872, -0.20412925373247692, 0.7026246253384617, 0.12201215332005758, 0.2283512250924352, 0.5687426140023165, -0.694235276118568, 0.0016878292574112436, -0.0003235471429102121, 0.0]
 * Inf-norm of residuals: 0.000000
 * Iterations: 3
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 4
 * Jacobian Calls (df/dx): 4

In [94]:
x = sys_solve.zero    

14-element Array{Float64,1}:
  1.0241741492535046   
 -0.13194828644853746  
  0.6695056054398789   
 -0.1283403666877176   
  0.6597414322426872   
 -0.20412925373247692  
  0.7026246253384617   
  0.12201215332005758  
  0.2283512250924352   
  0.5687426140023165   
 -0.694235276118568    
  0.0016878292574112436
 -0.0003235471429102121
  0.0                  

In [95]:
qg

0.05132763052538335

In [96]:
-v_g*cos(x[9])*x[6] + -v_g*sin(x[9])*x[5] 

0.050473026848537716

In [76]:
v_g*cos(x[9])*x[5] -v_g*sin(x[9])*x[6] -pg

1.3781531471579456e-11

In [ ]:
(ωb / cf)*(x[4] - x[6]) - ωg*ωb*x[1]

In [ ]:
    v_gd = v_g*cos(x[9])
    v_gq = -v_g*sin(x[9])

In [ ]:
v_gd

In [ ]:
v_gq